## Preparing the Kaggle test data

In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [189]:
data = pd.read_csv('../data/datasets/test.csv')

In [190]:
data.shape

(878, 80)

In [191]:
data.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [192]:
# lower case column names without spaces
data.columns = data.columns.str.lower().str.replace(' ', '_')

---

The next cells of code prepare the features in the Kaggle test data so that we can test the model predictions on these data and submit the predictions to the Kaggle competition. 

These preparation are essentially the feature scaling and contrast coding, as well as making sure that we remain only with the features included in the model fit to the training data set (in script 04_modeling).

At the end of these cells, we save the test data frame under the name 'test_df.csv' and we then should return to script number 04_modeling, at the end of it, to test the model predictions on the test_df data frame.

In [193]:
qual_dict = {'Ex' : 5,
             'Gd' : 4,
             'TA' : 3,
             'Fa' : 2,
             'Po' : 1}

In [194]:
data['exter_qual'] = data['exter_qual'].map(qual_dict)

In [195]:
data['kitchen_qual'] = data['kitchen_qual'].map(qual_dict)

In [196]:
data['pool_area'] = np.where(data['pool_area']==0, 0, 1)

In [197]:
data['bsmt_cond'] = data['bsmt_cond'].map(qual_dict)

In [198]:
data['fireplace_qu'] = data['fireplace_qu'].map(qual_dict)

In [199]:
data['garage_qual'] = data['garage_qual'].map(qual_dict)

In [200]:
data['garage_cond'] = data['garage_cond'].map(qual_dict)

In [201]:
data['heating_qc'] = data['heating_qc'].map(qual_dict)

In [202]:
style_dict = {'1Story' : 'story_1',
              '2Story' : 'story_2',
              '1.5Fin' : 'story_1_half_fin',
              'SLvl' : 'split_level',
              'SFoyer' : 'split_foyer',
              '2.5Unf' : 'story_2_half_unfin',
              '1.5Unf' : 'story_1_half_unfin',
              '2.5Fin' : 'story_2_half_fin'}

data['house_style'] = data['house_style'].map(style_dict)

In [203]:
data['garage_finish'].fillna('no_garage', inplace=True)

In [204]:
data['fence'].fillna('no_fence', inplace=True)

In [205]:
data['bsmtfin_type_1'].fillna('no_basement', inplace=True)

In [206]:
num_cols = [
    'overall_qual', 'overall_cond', 'year_remod/add', 'exter_qual', 'exter_cond', 'kitchen_qual', 'full_bath',
    'half_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'totrms_abvgrd', 'fireplaces', 'wood_deck_sf', 'pool_area',
    'misc_val', 'heating_qc', 'bsmt_cond', 'bsmtfin_sf_1', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath', 
    'fireplace_qu', 'garage_qual', 'garage_cond',

    # ID column
    'id'
]

In [207]:
cat_cols = [
    'house_style', 'heating', 'central_air', 'electrical', 'paved_drive', 
    'garage_finish', 'fence', 'bsmtfin_type_1']

In [208]:
data = data[num_cols + cat_cols]

In [209]:
data.shape

(878, 33)

---

In [210]:
# names of categorical features
cat_cols = [ 'house_style', 'heating', 'central_air', 'electrical', 'paved_drive', 'garage_finish', 'fence', 
    'bsmtfin_type_1', 'bsmt_cond', 'fireplace_qu', 'garage_qual', 'garage_cond', 'pool_area']

In [211]:
data['house_style'].astype('category').cat.reorder_categories(['split_foyer','split_level','story_1',
                                                               'story_1_half_unfin','story_1_half_fin',
                                                               'story_2','story_2_half_unfin','story_2_half_fin'],
                                                             inplace=True)

In [212]:
encoder = ce.BackwardDifferenceEncoder()
ce_house_style = encoder.fit_transform(data['house_style']).drop(columns='intercept')
ce_house_style.rename(columns = {
    'house_style_0' : 'house_style-split_level_VS_split_foyer',
    'house_style_1' : 'house_style-story_1_VS_split_level',
    'house_style_2' : 'house_style-story_1_half_unfin_VS_story_1',
    'house_style_3' : 'house_style-story_1_half_fin_VS_story_1_half_unfin',
    'house_style_4' : 'house_style-story_2_VS_story_1_half_fin',
    'house_style_5' : 'house_style-story_2_half_unfin_VS_story_2',
    'house_style_6' : 'house_style-story_2_half_fin_VS_story_2_half_unfin'
}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [213]:
data['heating'].value_counts()

GasA     866
GasW       7
Grav       4
Floor      1
Name: heating, dtype: int64

In [214]:
data['heating'].astype('category').cat.reorder_categories(['GasA','GasW','Grav','Floor'], inplace=True)

In [215]:
encoder = ce.BackwardDifferenceEncoder()
ce_heating = encoder.fit_transform(data['heating']).drop(columns='intercept')
ce_heating.rename(columns = {
    'heating_0' : 'heating-GasW_VS_GasA',
    'heating_1' : 'heating-Grav_VS_GasW',
    'heating_2' : 'heating-Floor_VS_Grav' }, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [216]:
data['central_air'].value_counts()

Y    823
N     55
Name: central_air, dtype: int64

In [217]:
data['central_air'].astype('category').cat.reorder_categories(['N','Y'], inplace=True)

In [218]:
encoder = ce.BackwardDifferenceEncoder()
ce_central_air = encoder.fit_transform(data['central_air']).drop(columns='intercept')
ce_central_air.rename(columns = {
    'central_air_0' : 'central_air_VS_no_central_air'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [219]:
data['electrical'].value_counts()

SBrkr    813
FuseA     48
FuseF     15
FuseP      1
Name: electrical, dtype: int64

In [220]:
data['electrical'].astype('category').cat.reorder_categories(['SBrkr','FuseA','FuseF','FuseP'], inplace=True)

In [221]:
encoder = ce.BackwardDifferenceEncoder()
ce_electrical = encoder.fit_transform(data['electrical']).drop(columns='intercept')
ce_electrical.rename(columns = {
    'electrical_0' : 'electrical-FuseA_VS_SBrkr',
    'electrical_1' : 'electrical-FuseF_VS_FuseA',
    'electrical_2' : 'electrical-FuseP_VS_FuseF'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [222]:
data['paved_drive'].value_counts()

Y    790
N     65
P     23
Name: paved_drive, dtype: int64

In [223]:
data['paved_drive'].astype('category').cat.reorder_categories(['N','P','Y'], inplace=True)

In [224]:
encoder = ce.BackwardDifferenceEncoder()
ce_paved_drive = encoder.fit_transform(data['paved_drive']).drop(columns='intercept')
ce_paved_drive.rename(columns = {
    'paved_drive_0' : 'paved_drive-P_VS_N',
    'paved_drive_1' : 'paved_drive-Y_VS_P'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [225]:
data['garage_finish'].value_counts()

Unf          382
RFn          233
Fin          218
no_garage     45
Name: garage_finish, dtype: int64

In [226]:
data['garage_finish'].astype('category').cat.reorder_categories(['no_garage','Unf','RFn','Fin'], inplace=True)

In [227]:
encoder = ce.BackwardDifferenceEncoder()
ce_garage_finish = encoder.fit_transform(data['garage_finish']).drop(columns='intercept')
ce_garage_finish.rename(columns = {
    'garage_finish_0' : 'garage_finish-Unf_VS_no_garage',
    'garage_finish_1' : 'garage_finish-RFn_VS_Unf',
    'garage_finish_2' : 'garage_finish-Fin_VS_RFn'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [228]:
data['fence'].value_counts()

no_fence    706
MnPrv       103
GdPrv        35
GdWo         32
MnWw          2
Name: fence, dtype: int64

In [229]:
data['fence'].astype('category').cat.reorder_categories(['no_fence','MnWw','GdWo','MnPrv','GdPrv'], inplace=True)

In [230]:
encoder = ce.BackwardDifferenceEncoder()
ce_fence = encoder.fit_transform(data['fence']).drop(columns='intercept')
ce_fence.rename(columns = {
    'fence_0' : 'fence-MnWw_VS_no_fence',
    'fence_1' : 'fence-GdWo_VS_MnWw',
    'fence_2' : 'fence-MnPrv_VS_GdWo',
    'fence_3' : 'fence-GdPrv_VS_MnPrv'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [231]:
data['bsmtfin_type_1'].value_counts()

Unf            248
GLQ            243
ALQ            136
Rec            105
BLQ             69
LwQ             52
no_basement     25
Name: bsmtfin_type_1, dtype: int64

In [232]:
data['bsmtfin_type_1'].astype('category').cat.reorder_categories(['no_basement','Unf','LwQ','Rec',
                                                                 'BLQ','ALQ','GLQ'], inplace=True)

In [233]:
encoder = ce.BackwardDifferenceEncoder()
ce_bsmtfin_type_1 = encoder.fit_transform(data['bsmtfin_type_1']).drop(columns='intercept')
ce_bsmtfin_type_1.rename(columns = {
    'bsmtfin_type_1_0' : 'bsmtfin_type_1-Unf_VS_no_basement',
    'bsmtfin_type_1_1' : 'bsmtfin_type_1-LwQ_VS_Unf',
    'bsmtfin_type_1_2' : 'bsmtfin_type_1-Rec_VS_LwQ',
    'bsmtfin_type_1_3' : 'bsmtfin_type_1-BLQ_VS_Rec',
    'bsmtfin_type_1_4' : 'bsmtfin_type_1-ALQ_VS_BLQ',
    'bsmtfin_type_1_5' : 'bsmtfin_type_1-GLQ_VS_ALQ'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [234]:
data['bsmt_cond'].value_counts()

3.0    781
2.0     39
4.0     33
Name: bsmt_cond, dtype: int64

In [235]:
data['bsmt_cond'] = ['4' if a==4 else '3' if a==3 else '2' if a==2 else 'no_basement' for a in data['bsmt_cond']]

In [236]:
data['bsmt_cond'].value_counts()

3              781
2               39
4               33
no_basement     25
Name: bsmt_cond, dtype: int64

In [237]:
data['bsmt_cond'].astype('category').cat.reorder_categories(['no_basement','2','3','4'], inplace=True)

In [238]:
encoder = ce.BackwardDifferenceEncoder()
ce_bsmt_cond = encoder.fit_transform(data['bsmt_cond']).drop(columns='intercept')
ce_bsmt_cond.rename(columns = {
    'bsmt_cond_0' : 'bsmt_cond-2_VS_no_basement',
    'bsmt_cond_1' : 'bsmt_cond-3_VS_2',
    'bsmt_cond_2' : 'bsmt_cond-4_VS_3'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [239]:
data['fireplace_qu'].value_counts()

4.0    220
3.0    193
2.0     16
1.0     15
5.0     12
Name: fireplace_qu, dtype: int64

In [240]:
data['fireplace_qu'] = ['5' if a==5 else '4' if a==4 else '3' if a==3 else '2' if a==2 else '1' if a==1 else 'no_fireplace' for a in data['fireplace_qu']]

In [241]:
data['fireplace_qu'].value_counts()

no_fireplace    422
4               220
3               193
2                16
1                15
5                12
Name: fireplace_qu, dtype: int64

In [242]:
data['fireplace_qu'].astype('category').cat.reorder_categories(['no_fireplace','1','2','3',
                                                                 '4','5'], inplace=True)

In [243]:
encoder = ce.BackwardDifferenceEncoder()
ce_fireplace_qu = encoder.fit_transform(data['fireplace_qu']).drop(columns='intercept')
ce_fireplace_qu.rename(columns = {
    'fireplace_qu_0' : 'fireplace_qu-1_VS_no_fireplace',
    'fireplace_qu_1' : 'fireplace_qu-2_VS_1',
    'fireplace_qu_2' : 'fireplace_qu-3_VS_2',
    'fireplace_qu_3' : 'fireplace_qu-4_VS_3',
    'fireplace_qu_4' : 'fireplace_qu-5_VS_4'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [244]:
data['garage_qual'].value_counts() 

3.0    782
2.0     42
4.0      6
1.0      3
Name: garage_qual, dtype: int64

In [245]:
data['garage_qual'] = ['4' if a==4 else '3' if a==3 else '2' if a==2 else '1' if a==1 else 'no_garage' for a in data['garage_qual']]

In [246]:
data['garage_qual'].value_counts() 

3            782
no_garage     45
2             42
4              6
1              3
Name: garage_qual, dtype: int64

In [247]:
data['garage_qual'].astype('category').cat.reorder_categories(['no_garage','1','2','3','4'], inplace=True)

In [248]:
encoder = ce.BackwardDifferenceEncoder()
ce_garage_qual = encoder.fit_transform(data['garage_qual']).drop(columns='intercept')
ce_garage_qual.rename(columns = {
    'garage_qual_0' : 'garage_qual-1_VS_no_garage',
    'garage_qual_1' : 'garage_qual-2_VS_1',
    'garage_qual_2' : 'garage_qual-3_VS_2',
    'garage_qual_3' : 'garage_qual-4_VS_3'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [249]:
data['garage_cond'].value_counts()

3.0    796
2.0     27
1.0      6
4.0      3
5.0      1
Name: garage_cond, dtype: int64

In [250]:
data['garage_cond'] = ['5' if a==5 else '4' if a==4 else '3' if a==3 else '2' if a==2 else '1' if a==1 else 'no_garage' for a in data['garage_cond']]

In [251]:
data['garage_cond'].value_counts()

3            796
no_garage     45
2             27
1              6
4              3
5              1
Name: garage_cond, dtype: int64

In [252]:
data['garage_cond'].astype('category').cat.reorder_categories(['no_garage','1','2','3',
                                                                 '4','5'], inplace=True)

In [253]:
encoder = ce.BackwardDifferenceEncoder()
ce_garage_cond = encoder.fit_transform(data['garage_cond']).drop(columns='intercept')
ce_garage_cond.rename(columns = {
    'garage_cond_0' : 'garage_cond-1_VS_no_garage',
    'garage_cond_1' : 'garage_cond-2_VS_1',
    'garage_cond_2' : 'garage_cond-3_VS_2',
    'garage_cond_3' : 'garage_cond-4_VS_3',
    'garage_cond_4' : 'garage_cond-5_VS_4'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [254]:
data['pool_area'].value_counts()

0    874
1      4
Name: pool_area, dtype: int64

In [255]:
data['pool_area'] = ['no_pool' if a==0 else 'pool' for a in data['pool_area']]

In [256]:
encoder = ce.BackwardDifferenceEncoder()
ce_pool_area = encoder.fit_transform(data['pool_area']).drop(columns='intercept')
ce_pool_area.rename(columns = {
    'pool_area_0' : 'pool_VS_no_pool'}, inplace=True)

C:\Users\Yair\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [257]:
# putting all the categorical features together
data_cat = pd.concat([ce_house_style, 
                      ce_heating, ce_central_air, ce_electrical, ce_paved_drive,
                      ce_garage_finish, ce_fence, ce_bsmtfin_type_1,
                     ce_bsmt_cond, ce_fireplace_qu, ce_garage_qual, ce_garage_cond, ce_pool_area], axis=1)

In [258]:
# numeric features to be scaled, except for the saleprice
num_cols = [
    'overall_qual', 'overall_cond', 'year_remod/add', 'exter_qual', 'exter_cond', 'kitchen_qual', 'full_bath',
    'half_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'totrms_abvgrd', 'fireplaces', 'wood_deck_sf',
    'heating_qc', 'bsmtfin_sf_1', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath']

In [259]:
data_num = data[num_cols]

In [260]:
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   overall_qual    878 non-null    int64 
 1   overall_cond    878 non-null    int64 
 2   year_remod/add  878 non-null    int64 
 3   exter_qual      878 non-null    int64 
 4   exter_cond      878 non-null    object
 5   kitchen_qual    878 non-null    int64 
 6   full_bath       878 non-null    int64 
 7   half_bath       878 non-null    int64 
 8   bedroom_abvgr   878 non-null    int64 
 9   kitchen_abvgr   878 non-null    int64 
 10  totrms_abvgrd   878 non-null    int64 
 11  fireplaces      878 non-null    int64 
 12  wood_deck_sf    878 non-null    int64 
 13  heating_qc      878 non-null    int64 
 14  bsmtfin_sf_1    878 non-null    int64 
 15  total_bsmt_sf   878 non-null    int64 
 16  bsmt_full_bath  878 non-null    int64 
 17  bsmt_half_bath  878 non-null    int64 
dtypes: int64(1

In [261]:
data_num['exter_cond'].value_counts()

TA    770
Gd     84
Fa     18
Ex      5
Po      1
Name: exter_cond, dtype: int64

In [262]:
data_num['exter_cond'] = [5 if a=='Ex' else 4 if a=='Gd' else 3 if a=='TA' else 2 if a=='Fa' else 1 for a in data_num['exter_cond']]

<ipython-input-262-ab95d8f7ddd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_num['exter_cond'] = [5 if a=='Ex' else 4 if a=='Gd' else 3 if a=='TA' else 2 if a=='Fa' else 1 for a in data_num['exter_cond']]


In [263]:
sc = StandardScaler()

In [264]:
data_num_Z = sc.fit_transform(data_num)

In [265]:
data_num_Z = pd.DataFrame(data_num_Z, columns=data_num.columns)

In [266]:
data_num_Z.shape

(878, 18)

In [267]:
data_cat.shape

(878, 48)

In [268]:
# putting the categorical and numeric features together, alongside the "ID" column for the Kaggle submission
data = pd.concat([data['id'], data_cat, data_num_Z], axis=1)

In [269]:
data.head()

,id,house_style-split_level_VS_split_foyer,house_style-story_1_VS_split_level,house_style-story_1_half_unfin_VS_story_1,house_style-story_1_half_fin_VS_story_1_half_unfin,house_style-story_2_VS_story_1_half_fin,house_style-story_2_half_unfin_VS_story_2,house_style-story_2_half_fin_VS_story_2_half_unfin,heating-GasW_VS_GasA,heating-Grav_VS_GasW,...,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,wood_deck_sf,heating_qc,bsmtfin_sf_1,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
0,2658,-0.875,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,-0.75,-0.5,...,1.349887,4.253806,1.595239,-0.924179,-0.771435,-0.135191,-1.007121,-0.034747,-0.830056,-0.24312
1,2718,0.125,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,-0.75,-0.5,...,3.758832,4.253806,2.221914,-0.924179,0.632217,-1.194990,-1.007121,2.257062,-0.830056,-0.24312
2,2414,-0.875,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,-0.75,-0.5,...,0.145415,-0.213708,0.341888,0.570166,0.054243,0.924608,0.257120,-0.920493,1.057998,-0.24312
3,1989,0.125,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,-0.75,-0.5,...,-1.059058,-0.213708,-0.911463,-0.924179,-0.771435,-1.194990,-1.007121,-0.160590,-0.830056,-0.24312
4,625,0.125,-0.75,-0.625,-0.5,-0.375,-0.25,-0.125,-0.75,-0.5,...,0.145415,-0.213708,-0.284788,2.064511,-0.771435,-0.135191,0.382631,0.870361,1.057998,-0.24312


In [270]:
# save the test_df data frame
data.to_csv('../data/test_df.csv', index=False)

---